# <center> Практическое задание по теме "Циклы"

<img src=https://1gai.ru/uploads/posts/2020-12/1608804716_54441.png width=300 align="right">

→ Поздравляем с освоением важных для анализа данных конструкций Python (переменные, структуры данных, условные операторы и циклы)!

Настало время промежуточной практики, чтобы закрепить все полученные в предыдущих модулях навыки! Мы будем применять их на реальном проекте. Сегодня мы обратимся к классике: займемся анализом текстов на примере «Войны и мира» Льва Николаевича Толстого!

В рамках практического кейса мы сначала вместе познакомимся с исходными данными и произведем некоторые манипуляции над ними. После чего вам предстоит самостоятельно выполнить несколько заданий на тему поиска наиболее значимых слов в тексте с помощью методов статистического анализа текста.

> Итак, приступим!

## <center> Знакомимся с данными

Текст произведения мы взяли в библиотеке [lib.ru](http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0073.shtml) и провели первоначальную обработку. Поскольку наша цель — обработка слов из этого произведения, мы разбили текст на слова и вывели каждое слово в отдельной строке. Кроме того, в местах, где начинаются главы, мы вывели строку `"[new chapter]"`.

> Исходный текстовый файл хранится в общем доступе и находится [здесь](https://raw.githubusercontent.com/SkillfactoryDS/Datasets/master/war_peace_processed.txt).

Для начала скачаем текст книги по ссылке.

In [1]:
# Импортируем библиотеку для выполнения HTTP-запросов в интернет
import requests

# Читаем текстовый файл по url-ссылке
data = requests.get("https://raw.githubusercontent.com/SkillfactoryDS/Datasets/master/war_peace_processed.txt").text

# Предобрабатываем текстовый файл
data = data.split('\n')
data.remove('')
data = data + ['[new chapter]']

# Выводим первые 100 слов из книги
print(data[:100])

['1', 'в', 'два', 'раза', 'короче', 'и', 'в', 'пять', 'раз', 'интереснее', '2', 'почти', 'нет', 'философических', 'отступлений', '3', 'в', 'сто', 'раз', 'легче', 'читать', 'весь', 'французский', 'текст', 'заменен', 'русским', 'в', 'переводе', 'самого', 'толстого', '4', 'гораздо', 'больше', 'мира', 'и', 'меньше', 'войны', '5', 'хеппи-энд', 'эти', 'слова', 'я', 'поместил', 'семь', 'лет', 'назад', 'на', 'обложку', 'предыдущего', 'издания', 'указав', 'в', 'аннотации', 'первая', 'полная', 'редакция', 'великого', 'романа', 'созданная', 'к', 'концу', '1866', 'года', 'до', 'того', 'как', 'толстой', 'переделал', 'его', 'в', '1867--1869', 'годах', '--', 'и', 'что', 'я', 'использовал', 'такие-то', 'публикации', 'думая', 'что', 'все', 'всё', 'знают', 'я', 'не', 'объяснил', 'откуда', 'взялась', 'эта', 'первая', 'редакция', 'я', 'оказался', 'неправ', 'и', 'в', 'результате', 'оголтелые', 'и']


## <center> Работаем с данными

*Для* начала найдем общее количество слов и количество уникальных слов в тексте

In [2]:
# Превращаем список в множество, удаляя дублирующиеся слова
word_set = set(data)
# Удаляем из множества слово, символизирующее раздел между главами
word_set.discard('[new chapter]')
# Выводим результаты
print('Общее количество слов: {}'.format(len(data)))
print('Общее количество уникальных слов: {}'.format(len(word_set)))

Общее количество слов: 300080
Общее количество уникальных слов: 38210


Давайте напишем программу, которая посчитает частоту каждого слова. Для этого создадим словарь, ключами которого будут являться слова, а значения - количество вхождений этого слова в текст произведения. Заодно подсчитаем количество глав

In [3]:
# Инициализируем пустой словарь
word_counts = {}
# Инициализируем количество глав
count_chapter = 0
# Создаем цикл по всем словам из списка слов
for word in data:
    # Проверяем, что текущее слово - обозначение новой главы
    if word == '[new chapter]':
        # Если условие выполняется, то увеличиваем количество глав на 1
        count_chapter += 1
        # Переходим на новую итерацию цикла
        continue
    # Проверяем, что текущего слова еще нет в словаре слов
    if word not in word_counts:
        # Если условие выполняется, инициализируем новый ключ 1
        word_counts[word] = 1
    else:
        # В противном случае, увеличиваем количество слов на 1
        word_counts[word] += 1

# Выводим количество глав
print('Количество глав: {}'.format(count_chapter))

# Создаем цикл по ключам и их порядковым номерам полученного словаря
for i, key in enumerate(word_counts):
    # Выводим только первые 10 слов
    if i == 10:
        break
    print(key, word_counts[key])

Количество глав: 171
1 10
в 6997
два 215
раза 35
короче 3
и 14592
пять 61
раз 296
интереснее 4
2 12


Разделим все слова на главы. Для этого создадим список, в котором будем хранить списки - слова из определенной главы.

In [4]:
# Инициализируем общий список, в котором будем хранить списки слов в каждой главе
chapter_data = []
# Инициализируем список слов, в котором будет хранить слова одной главы
chapter_words = []

# Создаем цикл по всем словам из списка
for word in data:
    # Проверяем, что текущее слово - обозначение новой главы
    if word == '[new chapter]':
        # Если условие выполняется, добавляем список со словами из главы в общий список
        chapter_data.append(chapter_words)
        # Обновляем (перезаписываем) список со словами из текущей главы
        chapter_words = []
    else:
        # В противном случае, добавляем текущее слово в список со словами из главы
        chapter_words.append(word)

# Проверяем, что у нас получилось столько же списков, сколько глав в произведении
print('Вложенный список содержит {} внутренних списка'.format(len(chapter_data)))
# Выведем первые 100 слов 0-ой главы
print(chapter_data[0][:100])

Вложенный список содержит 171 внутренних списка
['1', 'в', 'два', 'раза', 'короче', 'и', 'в', 'пять', 'раз', 'интереснее', '2', 'почти', 'нет', 'философических', 'отступлений', '3', 'в', 'сто', 'раз', 'легче', 'читать', 'весь', 'французский', 'текст', 'заменен', 'русским', 'в', 'переводе', 'самого', 'толстого', '4', 'гораздо', 'больше', 'мира', 'и', 'меньше', 'войны', '5', 'хеппи-энд', 'эти', 'слова', 'я', 'поместил', 'семь', 'лет', 'назад', 'на', 'обложку', 'предыдущего', 'издания', 'указав', 'в', 'аннотации', 'первая', 'полная', 'редакция', 'великого', 'романа', 'созданная', 'к', 'концу', '1866', 'года', 'до', 'того', 'как', 'толстой', 'переделал', 'его', 'в', '1867--1869', 'годах', '--', 'и', 'что', 'я', 'использовал', 'такие-то', 'публикации', 'думая', 'что', 'все', 'всё', 'знают', 'я', 'не', 'объяснил', 'откуда', 'взялась', 'эта', 'первая', 'редакция', 'я', 'оказался', 'неправ', 'и', 'в', 'результате', 'оголтелые', 'и']


In [5]:
chapter_data[15][100]

'в'

Подсчитаем, сколько раз каждое слово встречается в каждой из глав

In [6]:
# Инициализируем список, в котором будем хранить словари
chapter_words_count = []

# Создаем цикл по элементам внешнего списка со словами
for chapter_words in chapter_data:
    # Инициализируем пустой словарь, куда будем добавлять результаты
    temp = {}
    # Создаем цикл по элементам внутреннего списка
    for word in chapter_words:
        # Проверяем, что текущего слова еще нет в словаре
        if word not in temp:
            # Если условие выполняется, добавляем ключ в словарь
            temp[word] = 1
        else:
            # В противном случае, увеличиваем количество влождений слова в главу
            temp[word] += 1
    # Добавляем получившийся словарь в список
    chapter_words_count.append(temp)

# Выводим результат
#print(chapter_words_count)

In [7]:
chapter_words_count[15]['князю']

2

In [8]:
# Создаем цикл по ключам словаря - спискам слов и их порядковым номерам
for chapter_number, chapter_dict in enumerate(chapter_words_count):
    # Выводим только первые 5 глав
    if chapter_number == 5:
        break
    # Выводим номер главы
    print('-' * 40)
    print('Chapter: {}'.format(chapter_number))
    print('-' * 40)
    # Создаем цикл по ключам - словам и их порядковым номерам
    for j, word in enumerate(chapter_dict):
        # Выводим первые 10 слов из главы
        if j == 10:
            break
        print(word, chapter_dict[word])

----------------------------------------
Chapter: 0
----------------------------------------
1 1
в 37
два 3
раза 1
короче 1
и 34
пять 1
раз 2
интереснее 1
2 1
----------------------------------------
Chapter: 1
----------------------------------------
автора 1
я 20
пишу 2
до 1
сих 1
пор 1
только 5
о 2
князьях 1
графах 1
----------------------------------------
Chapter: 2
----------------------------------------
первая 1
----------------------------------------
Chapter: 3
----------------------------------------
-- 81
ну 5
что 44
князь 21
генуя 1
и 94
лукка 1
стали 1
не 57
больше 2
----------------------------------------
Chapter: 4
----------------------------------------
гостиная 1
анны 2
павловны 2
начала 1
понемногу 1
наполняться 1
приехала 3
высшая 1
знать 1
петербурга 2


Давайте резюмировать, что мы с вами уже получили:

* `word_set` - множество из всех слов, которые есть в книге

* `count_chapter` - количество глав в книге (171)

* `word_counts` - словарь, ключами которого являются слова, а значениями - количество вхождений этих слов в книгу

* `chapter_data` - список из 171 списка, где элементы вложенных списков - все слова из главы. Каждый список соответствует своей главе

* `chapter_words_count` - список из 171 словаря, где ключи - слова, а значения - количество слов в главе. Каждый словарь соответствует своей главе

Учтите, что эти данные могут пригодиться вам при выполнении дальнейших заданий.

> А теперь к заданиями!

## <center> Задания для самостоятельного решения

### Задание 1.

Давайте введем понятие частоты употребления отдельного слова в документе (`term frequency`, или `tf`). В нашем случае речь идёт не о документах, а о главах книги (выше мы писали, что в текстовом документе главы разделяются строкой '[new chapter]').

Формула для вычисления `term frequency` для слова `word`:
$$ tf_{word, chapter} = \frac {n_{word, chapter}} {n_{chapter}}$$

где
* ${n_{word, chapter}}$ - сколько раз слово `word` встречается в главе `chapter`,
* $n_{chapter}$ - количество слов в главе `chapter`.


Например, слово `"гостья"` употребляется в 15-ой главе 10 раз (${n_{word, chapter}}$).(кстати, главы у нас нумеруются с 0). Общее количество слов в тексте 15-ой главы - 1359 ($n_{chapter}$). Тогда:

$$ tf_{гостья, 15} = \frac{10}{1359} \approx 0.007358$$

**Задание:**

Напишите программу, которая позволит получать частоту употребления любого заданного слова `target_word` в заданной главе `target_chapter`.

**Дополнительное требование:**

*Пострайтесь сделать программу максимально обобщенной. То есть желательно рассчитать характеристику `tf` для всех слов из каждой главы, чтобы впоследствии не было необходимости производить вычисления снова.*

**Подсказка:**

*Для этого вы можете для каждой главы создать словарь, ключами которого являются слова, а значения - частота употребления этого слова в этой главе*

**Протестируйте работу программы на нескольких словах и главах.**

In [9]:
target_word = 'гостья'
target_chapter = 15

for chapter_number, chapter_dict in enumerate(chapter_words_count):
    if chapter_number == target_chapter:#Ищем tf только в заданной главе
    # Выводим номер главы
      print('-' * 40)
      print('Chapter: {}'.format(chapter_number))
      print('-' * 40)
      # Считаем частоту употребления нужного слова
      tf = round(chapter_words_count[target_chapter][target_word] /sum(chapter_dict.values()), 6)
      x=sum(chapter_dict.values())#Для красивого вывода общего количества слов в главе
print('Количество слов "{}" в главе "{}": {}'.format(target_word, target_chapter,chapter_words_count[target_chapter][target_word]))
print('Общее количество слов в главе "{}": {}'.format(target_chapter, x))
print('Частота употребления слова "{}" в главе "{}":{}'.format(target_word, target_chapter, tf))


----------------------------------------
Chapter: 15
----------------------------------------
Количество слов "гостья" в главе "15": 10
Общее количество слов в главе "15": 1359
Частота употребления слова "гостья" в главе "15":0.007358


### Задание 2.

Пришло время познакомиться с понятием `document frequency`.

`Document frequency` (для удобства сократим до `df`) — это доля документов, в которых встречается искомое слово.

Вычисляется по формуле:
df_word =N_word/N,

где
* N_word - число документов (глав) содержащих слово `word`,
* N - общее число документов (глав).

Объясним на примере: наш текст состоит из 171 главы ($N$), а слово `"человек"` встречается в 115 главах. Тогда:

$$ df_{человек} = \frac{115}{171} \approx 0.6725$$

**Задание:**

Напишите программу, которая позволит вычислять document frequency для заданного слова `target_word` и выведить результат на экран.

**Дополнительное требование:**

*Пострайтесь сделать программу максимально обобщенной. То есть желательно рассчитать характеристику `df` для всех уникальных слов из книги, чтобы впоследствии не было необходимости производить вычисления снова.*

**Подсказка:**
*Для этого вы можете создать словарь, ключами которого являются слова из книги, а значения - доля документов, содержащих эти слова*

**Протестируйте работу программы на нескольких словах**

In [10]:
target_word = 'человек'

N_word=0
# Ваш код здесь
for chapter_dict in chapter_words_count:
    for words in chapter_dict:
      if words == target_word:
        N_word+=1
        break
N=chapter_number+1
df=round(N_word/(N), 4)
print('Количество глав: {}'.format(N))
print('В скольких главах встречается слово "{}": {}'.format(target_word, N_word))
print('Доля документов, в которых встречается слово "{}": {}'.format(target_word, df))


Количество глав: 171
В скольких главах встречается слово "человек": 115
Доля документов, в которых встречается слово "человек": 0.6725


### Задание 3

Пришло время дать разъяснения: для чего мы делали вычисления выше и что нас ждет впереди?

> Если какое-то слово часто употребляется в документе, то, вероятно, этот документ что-то рассказывает о предмете/действии, описываемом этим словом. Скажем, если вы читаете книгу, в которой много раз употребляется слово `"заяц"`, то, вероятно, эта книга про зайцев.

> Однако, если вы возьмёте слово `"и"`, то оно будет встречаться почти в каждой книге много раз.

Таким образом, если мы хотим найти наиболее значимые слова в книге, мы, с одной стороны, хотим найти наиболее частые слова, а с другой — убрать те, которые не несут важной информации, так как встречаются везде.

Такая задача хорошо решается с помощью `tf-idf` — статистической метрики для оценки важности слова в тексте. Другими словами, `tf-idf` — это «контрастность» слова в документе (насколько оно выделяется среди других слов).

Формула для вычисления следующая:

`tf-idf = term frequency * inverse document frequency`

* `tf` — это частотность термина, которая измеряет, насколько часто термин встречается в документе.

* `idf` — это обратная документная частотность термина. Она измеряет непосредственно важность термина во всём множестве документов.

Чтобы получить `idf`, необходимо поделить 1 на полученную в Задании 2 документную частоту (`df`):

$$idf = \frac{1}{df}$$

Мы будем использовать не сырые значения `idf`, а их логарифмы, то есть $tf * log(idf)$. Сейчас мы не будем заострять внимания на том, почему следует использовать именно логарифм — это долгий разговор. Вернемся к нему, когда будем изучать методы машинного обучения для обработки текстов. Подробнее о `tf-idf` вы можете почитать [здесь](https://translated.turbopages.org/proxy_u/en-ru.ru.15518a02-63e76541-6895b80b-74722d776562/https/www.freecodecamp.org/news/how-to-process-textual-data-using-tf-idf-in-python-cd2bbc0a94a3/).

В качестве примера измерим `tf-idf` слова `"анна"` в главе 4. Слово `"анна"` встречается в указанной главе 7 раз, при этом в 4 главе 1060 слов, всего же слово `"анна"` упоминается в 32 главах из 171.

Таким образом, `tf-idf` данного слова в данной главе будет равно:

$$tf\_idf_{анна, 4} = tf * log(\frac{1}{df}) = \frac{7}{1060} * log(\frac {171}{32}) \approx 0.011067$$

**Примечание:** здесь используется натуральный логарифм по основанию $e$, однако в общем случае основание логарифма не имеет значения, так как характеристика `tf-idf` используется для сравнения контрастности слов между собой

**Задание**:

Напишите программу, которая позволяет вычислять значение `tf-idf` для заданного слова `target_word` в заданной главе `target_chapter`.

**Дополнительное требование:**

*Пострайтесь сделать программу максимально оптимальной. То есть желательно рассчитать характеристику `tf-idf` для всех слов из каждой главы книги, чтобы впоследствии не было необходимости производить вычисления снова.*

**Протестируйте работу программы на нескольких словах и главах.**

**Примечание:**

Натуральный логарифм можно вычислить с помощью функции [log](https://pythonim.ru/chisla/funktsiya-log-v-python) из встроенного в Python модуля math:


In [11]:
# Импортируем функцию log из модуля math:
from math import log
print(log(18))

2.8903717578961645


**Примечание.**

**Модуль (библиотека) в Python** — это любой программный файл, который содержит в себе код, включая функции. В нашем случае math — это встроенный модуль, содержащий функционал для математических вычислений. Подробнее о math вы можете почитать [здесь](https://pythonworld.ru/moduli/modul-math.html).

In [12]:
from math import log
target_word = 'анна'
target_chapter = 4

# Ваш код здесь
N_word=0

for chapter_number, chapter_dict in enumerate(chapter_words_count):
    if chapter_number == target_chapter:
    # Выводим номер главы
      print('-' * 40)
      print('Chapter: {}'.format(chapter_number))
      print('-' * 40)
      # Cчитаем частоту употребления нужного слова
      tf = round(chapter_words_count[target_chapter][target_word] /sum(chapter_dict.values()), 6)
      x=sum(chapter_dict.values())#Для красивого вывода общего количества слов в главе
    for words in chapter_dict:
      if words == target_word:
        N_word+=1
        break
N=chapter_number+1
df=round(N_word/(N), 4)
tf_idf=round(tf*log(1/df), 6)
print('Частота употребления слова "{}" в главе "{}":{}'.format(target_word, target_chapter, tf))
print('Доля документов, в которых встречается слово "{}": {}'.format(target_word, df))
print('Важность слова {}: {}'.format(target_word,tf_idf))

----------------------------------------
Chapter: 4
----------------------------------------
Частота употребления слова "анна" в главе "4":0.006604
Доля документов, в которых встречается слово "анна": 0.1871
Важность слова анна: 0.011069


### Задание 4.

Теперь, когда мы умеем вычислять `tf-idf` для каждого слова в главе, мы можем найти те слова, которые являются самыми «контрастными» для данной главы, то есть они могут являться в своём роде заголовком для главы.

Например, для главы 3 наиболее значимыми словами будут:

`"анна", "павловна", "функе"`

**Задание:**

Напишите программу, которая позволяет вывести три слова, имеющие самое высокое значение `tf-idf` в заданной главе `target_chapter` в порядке убывания `tf-idf`.

In [13]:
target_chapter = 3

# Ваш код здесь
tf_list={}

df_list_count={}
df_list={}
df_sort={}

tf_idf_list={}
tf_idf_sort={}
count_word_in_tf_list=0
# Создаем цикл по ключам словаря - спискам слов и их порядковым номерам
for chapter_number, chapter_dict in enumerate(chapter_words_count):

    if chapter_number == target_chapter:
    # Выводим номер главы
      print('-' * 40)
      print('Chapter: {}'.format(chapter_number))
      print('-' * 40)
#Ищем tf------------------------------------------------------------------------
      X=sum(chapter_dict.values())#Переменная для сохранения всех слов в определенной главе
      for word in chapter_dict:
        tf=round(chapter_words_count[target_chapter][word]/X, 6)
        tf_list.update({word: tf})
#-------------------------------------------------------------------------------
#Создали дополительный словарь для вычисления (N_word как в прошлых заданиях) сколько раз
#слова из tf_list(словарь для вычисления tf именно в нужной главе) встречаются в других главах
    for i, word in enumerate(chapter_dict):
      if word in tf_list:
        if word in df_list_count:
          df_list_count.update({word:df_list_count[word]+1})
        else:
          df_list_count.update({word:1})
#Ищем df, путем разделения значений из словаря df_list_count на общее количетва глав
for word in df_list_count:
  df=round(df_list_count[word]/(N), 4)
  df_list.update({word:df})
#Итоговый список для высчисления tf_idf-----------------------------------------
for words in tf_list:
  tf_idf=round(tf_list[words]*log(1/df_list[words]), 6)
  tf_idf_list.update({words:tf_idf})


#Сортировка и вывод 3 важных слов------------------------------------------------

tf_idf_sort=dict(sorted(tf_idf_list.items(), reverse=True,  key=lambda item: item[1]))
print(*list(tf_idf_sort.keys())[:3], sep=', ')
print()
print('tf :', tf_list)

print('df :', df_list)
print('tf_idf сортированная:' , tf_idf_sort)

----------------------------------------
Chapter: 3
----------------------------------------
павловна, анна, функе

tf : {'--': 0.036486, 'ну': 0.002252, 'что': 0.01982, 'князь': 0.009459, 'генуя': 0.00045, 'и': 0.042342, 'лукка': 0.00045, 'стали': 0.00045, 'не': 0.025676, 'больше': 0.000901, 'как': 0.012162, 'поместья': 0.000901, 'фамилии': 0.000901, 'буонапарте': 0.002252, 'нет': 0.001802, 'я': 0.013964, 'вам': 0.001802, 'вперед': 0.00045, 'говорю': 0.00045, 'если': 0.001802, 'вы': 0.010811, 'мне': 0.003604, 'скажете': 0.00045, 'у': 0.006757, 'нас': 0.001802, 'война': 0.00045, 'позволите': 0.00045, 'себе': 0.001351, 'защищать': 0.00045, 'все': 0.006306, 'гадости': 0.00045, 'ужасы': 0.00045, 'этого': 0.001802, 'антихриста': 0.00045, 'право': 0.000901, 'верю': 0.001802, 'он': 0.013964, 'антихрист': 0.00045, 'вас': 0.004505, 'знаю': 0.000901, 'уже': 0.001802, 'друг': 0.000901, 'мой': 0.001351, 'верный': 0.000901, 'раб': 0.001351, 'говорите': 0.000901, 'здравствуйте': 0.000901, 'вижу': 0